In [23]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import os

'/content'

In [147]:
PATIENT_PATH = str(os.getcwd()) + "/dataset/schizoprenia/"
HEALTHY_PATH = str(os.getcwd()) + "/dataset/healthy/"
patient_data = [ PATIENT_PATH + f"patient_{i}.csv" for i in range(0,30)]
healthy_data = [ HEALTHY_PATH + f"healthy{i}.csv" for i in range(0,30)]
patient_df = [pd.read_csv(f"{i}") for i in patient_data]
healthy_df = [pd.read_csv(f"{i}") for i in healthy_data]

In [148]:
for i in range(30):
  patient_df[i].pop('Unnamed: 0')
  healthy_df[i].pop('Unnamed: 0')

In [149]:
def FFT(df):
    fs = 128 # Sampling rate (128 Hz)
    band_data = pd.DataFrame() 
    for index in list(df.index):
        data = df.loc[index]  # 2 sec of data b/w 0.0-100.0

        # Get real amplitudes of FFT (only in postive frequencies)
        fft_vals = np.absolute(np.fft.rfft(data))
        # Get frequencies for amplitudes in Hz
        fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
        # Define EEG bands
        eeg_bands = {'Delta': (0, 4),
                     'Theta': (4, 8),
                     'Alpha': (8, 12),
                     'Beta': (12, 30),
                     'Gamma': (30, 45)}

        # Take the mean of the fft amplitude for each EEG band
        eeg_band_fft = dict()
        for band in eeg_bands:  
            freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
            eeg_band_fft[band] = np.mean(fft_vals[freq_ix])

        ser = pd.Series(eeg_band_fft,name=index)
        band_data = band_data.append(ser)
    return band_data

In [150]:
patient_fft = [FFT(i) for i in patient_df]
healthy_fft = [FFT(j) for j in healthy_df]

In [151]:
patient_fft[29].head()

,Alpha,Beta,Delta,Gamma,Theta
0,27262.162707,14945.249661,59716.581779,3463.703027,33103.883420
1,33338.745594,18190.503873,78832.351322,3835.470285,41360.646744
2,32285.493901,18053.385476,71223.192899,3848.840391,38647.153017
3,26783.839870,15042.239384,53165.976531,3456.802018,30536.181589
4,28594.914934,16840.642900,47448.171918,4052.956331,33115.647404


In [152]:
healthy_fft[29].head()

,Alpha,Beta,Delta,Gamma,Theta
0,29974.230338,11944.755217,57566.578124,2384.948480,37962.437435
1,36474.278252,15187.213331,83390.120470,2723.596946,45332.440774
2,38312.276470,15379.847262,78054.316109,2669.135900,47787.220585
3,37052.597533,13903.965016,75207.905342,2728.630558,42739.439797
4,28430.196906,10839.878122,63807.288448,2256.639311,37382.900448


In [168]:
feature_names = ["Alpha","Beta","Delta","Gamma","Theta"]
patient_tensor = [] #np.array([tf.convert_to_tensor(f"{patient_fft[i][feature_names]}") for i in range(len(patient_fft))])
healthy_tensor = [] #np.array([tf.convert_to_tensor(f"{healthy_fft[i][feature_names]}") for i in range(len(healthy_fft))])

In [169]:
for i in range(30):
  patient_tensor.append(tf.convert_to_tensor(patient_fft[i][feature_names]))
  healthy_tensor.append(tf.convert_to_tensor(healthy_fft[i][feature_names]))

In [183]:
patient_tensor

[<tf.Tensor: shape=(16, 5), dtype=float64, numpy=
 array([[ 42692.99350169,  15103.20443975,  88473.3121102 ,
           3880.28260395,  55976.57836203],
        [ 41736.95092603,  14636.46320541,  97836.52650364,
           3876.51740387,  58941.91743774],
        [ 42715.94770905,  15751.89983962,  99231.10959932,
           4339.46824446,  56509.78070492],
        [ 38186.46004139,  15248.16618262,  81601.65784637,
           4414.48116928,  51678.25463373],
        [ 45563.64190154,  14794.0407036 ,  74862.18347763,
           3750.27316992,  54752.42937769],
        [ 57795.06769198,  16206.51285496,  79101.01338845,
           4009.30194298,  66253.3969412 ],
        [ 64637.652099  ,  16842.21665176,  89212.18808954,
           4310.70365232,  76637.29792452],
        [ 59319.28574912,  16739.30174791,  81958.8817801 ,
           4236.04655335,  71836.95578229],
        [ 45960.73502146,  17174.16929763,  72679.84656247,
           4994.72354531,  58794.13519204],
        [ 4871

In [175]:
model = models.Sequential()
model.add(layers.Conv1D(16, (3), activation='relu', input_shape=(16, 5)))
model.add(layers.MaxPooling1D((2)))
model.add(layers.Conv1D(32, (3), activation='relu'))
model.add(layers.MaxPooling1D((2)))

In [176]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 14, 16)            256       
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 7, 16)            0         
 g1D)                                                            
                                                                 
 conv1d_22 (Conv1D)          (None, 5, 32)             1568      
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 2, 32)            0         
 g1D)                                                            
                                                                 
Total params: 1,824
Trainable params: 1,824
Non-trainable params: 0
_________________________________________________________________


In [178]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

In [179]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_21 (Conv1D)          (None, 14, 16)            256       
                                                                 
 max_pooling1d_14 (MaxPoolin  (None, 7, 16)            0         
 g1D)                                                            
                                                                 
 conv1d_22 (Conv1D)          (None, 5, 32)             1568      
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 2, 32)            0         
 g1D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 64)              